# Homework 1 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLSfDjWeeZJw5EvmKn1x_6b9xicjn7ed3MF0rbNm4Cmwr7psSkQ/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("wikipedia_train.csv")
test = pd.read_csv("wikipedia_test.csv")

## Data Analysis

### Количество рууских статей в Википедии

In [3]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [4]:
get_language(train['Page'][5])

'en'

In [5]:
lang = train['Page'].apply(lambda x: get_language(x))

In [6]:
train['lang'] = lang

In [7]:
lang.value_counts()

en    140
fr    128
ja    124
na    124
de    122
ru    102
es    101
zh     98
Name: Page, dtype: int64

### Самая популярная статья

In [8]:
train.fillna(0, inplace=True)

In [9]:
train_ru = train[train['lang']=='ru']
train_ru = train_ru.drop(['lang','Page'], axis = 1)
train_ru.fillna(0, inplace = True)
train_ru['means'] = train_ru.apply(np.mean, axis = 1)

In [12]:
train_ru['means']==train_ru['means'].max()

6      False
8      False
38     False
154    False
180     True
548    False
602    False
639    False
640    False
641    False
642    False
643    False
644    False
645    False
646    False
647    False
648    False
649    False
650    False
651    False
652    False
653    False
654    False
655    False
656    False
657    False
658    False
659    False
660    False
661    False
       ...  
704    False
705    False
706    False
707    False
708    False
709    False
710    False
711    False
712    False
713    False
714    False
715    False
716    False
717    False
718    False
719    False
720    False
721    False
722    False
723    False
724    False
725    False
726    False
727    False
728    False
729    False
730    False
731    False
732    False
733    False
Name: means, Length: 102, dtype: bool

In [11]:
train.iloc[180,:1]

Page    Facebook_ru.wikipedia.org_desktop_all-agents
Name: 180, dtype: object

In [324]:
train_ru.index

Int64Index([  6,   8,  38, 154, 180, 548, 602, 639, 640, 641,
            ...
            724, 725, 726, 727, 728, 729, 730, 731, 732, 733],
           dtype='int64', length=102)

## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [13]:
train_melt = pd.melt(train, value_vars=train.columns[1:], id_vars=['Page'])

In [14]:

train_melt.head()

,Page,variable,value
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [15]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Что б оценить прогноз нужно представить `test` данные в следующем формате:

In [16]:
test_melt = pd.melt(test, value_vars=test.columns[1:], id_vars=['Page'])

In [17]:
test_melt.shape

(58218, 3)

И затем вызвать функцию для подсчета метрики.

### Last day baseline

### На основе одного дня

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

In [18]:
train_day = train.iloc[:,428]

In [19]:
test_pred = test.copy(deep = True)
for i in test.columns[1:]:
    test_pred[i]=train_day

In [21]:
test_pred = pd.melt(test_pred, value_vars=test_pred.columns[1:], id_vars=['Page'])

In [24]:
test_melt['pred_Visits'] = test_pred['value']

In [25]:
test_melt.rename(columns = {'value':'Visits'}, inplace =True)

In [26]:
pandas_smape(test_melt)

54.16127748085736

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (воспользуйтесь функцией [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

In [79]:
train_30 = train.iloc[:,399:429]
train_30.fillna(0, inplace=True)

In [81]:
train_30.shape

(939, 30)

In [82]:
test_pred_30 = pd.DataFrame({i:np.median(train_30, axis = 1) for i in test.columns[1:]})

😏

In [83]:
test_pred_30 = pd.melt(test_pred_30, value_vars=test_pred_30.columns, id_vars=['Page'])

In [84]:
test_pred_30.rename(columns={'value':'pred_Visits'}, inplace = True)

In [85]:
test_melt.rename(columns = {'value':'Visits'}, inplace = True)

In [86]:
test_melt['pred_Visits']= test_pred_30['pred_Visits']

In [87]:
test_melt = test_melt.drop('SMAPE', axis = 1)

In [88]:
test_melt.head()

,Page,variable,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43.0,24.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4.0,2.5
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15.0,15.5
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101.0,110.5


In [89]:
pandas_smape(test_melt)

51.89889768202033

### Модель с учетом выходных и разными окнами медианы

In [39]:
train_week = train.copy(deep = True)

In [40]:
train_week = train_week.drop(['Page','lang'], axis = 1 )

In [41]:
train_week.columns= pd.to_datetime(train_week.columns).dayofweek.values

In [42]:
train_week.columns = [1 if i>=0 and i<=4 else 0 for i in train_week.columns]

In [43]:
train_week = train_week.groupby(train_week.columns, axis = 1).median()

In [44]:
test_week = test.drop('Page', axis = 1)

In [45]:
test_week.columns  = pd.to_datetime(test_week.columns).dayofweek.values

In [48]:
test_week.columns = [1 if i>=0 and i<=4 else 0 for i in test_week.columns]

In [50]:
for i in test_week.columns:
    test_week[i]= train_week[i]

In [51]:
test_week.head()

,0,0,1,1,1,1,1,0,0,1,...,1,1,1,1,0,0,1,1,1,1
0,26.0,26.0,29.0,29.0,29.0,29.0,29.0,26.0,26.0,29.0,...,29.0,29.0,29.0,29.0,26.0,26.0,29.0,29.0,29.0,29.0
1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,12.5,12.5,11.5,11.5,11.5,11.5,11.5,12.5,12.5,11.5,...,11.5,11.5,11.5,11.5,12.5,12.5,11.5,11.5,11.5,11.5
4,63.0,63.0,93.0,93.0,93.0,93.0,93.0,63.0,63.0,93.0,...,93.0,93.0,93.0,93.0,63.0,63.0,93.0,93.0,93.0,93.0


In [52]:
test_week['Page'] = test['Page']

In [53]:
test_week_melt = pd.melt(test_week, value_vars=test_week.columns[:-1], id_vars=['Page'])

In [54]:
test_week_melt.head()

,Page,variable,value
0,15._November_de.wikipedia.org_desktop_all-agents,0,26.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,0,2.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,0,12.5
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,0,63.0


In [55]:
test_melt.rename(columns = {'value':'Visits'}, inplace = True)

In [56]:
test_melt['pred_Visits']= test_week_melt['value']

In [57]:
test_melt = test_melt.drop('SMAPE', axis = 1)

In [59]:
test_melt.head()

,Page,variable,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43.0,26.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4.0,2.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15.0,12.5
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101.0,63.0


In [60]:
pandas_smape(test_melt)

52.41218109833793

### Перебор окна медиан

Выходной или не выходной

In [90]:
import warnings
warnings.filterwarnings("ignore")

In [96]:
train = pd.read_csv("wikipedia_train.csv")

train.fillna(0, inplace=True)

In [111]:

smape_results = pd.DataFrame({i:np.ones(61)})

for j in range(1,61):
    test_pred = pd.DataFrame({i:np.median(train.iloc[:,((-1)*j):], axis = 1) for i in test.columns[1:]})

    test_pred['Page'] = test.Page
    test_predm = pd.melt(test_pred, value_vars=test_pred.columns[:-1], id_vars='Page')

    testm = pd.melt(test, value_vars=test.columns[1:], id_vars='Page')
    testm['pred_Visits'] = test_predm.value
    testm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

    smape_results.iloc[j,0] = pandas_smape(testm)
smape_results  

,0
0,1.000000
1,54.161277
2,52.605790
3,51.535987
4,51.327896
5,50.703628
6,50.676091
7,50.312226
8,50.096783
9,49.842494


In [115]:
from datetime import datetime, date, time

train_weekday_columns = np.array([datetime.strptime(i, "%Y-%m-%d").isoweekday() for i in train.columns[1:]])
train_workday = train.iloc[:, (train_weekday_columns != 6)&(train_weekday_columns != 7)]
train_weekend = train.iloc[:, (train_weekday_columns == 6)|(train_weekday_columns == 7)]

test_weekday_columns = np.array([datetime.strptime(i, "%Y-%m-%d").isoweekday() for i in test.columns[1:]])
test_workday = test.iloc[:, (test_weekday_columns != 6)&(test_weekday_columns != 7)]
test_weekend = test.iloc[:, (test_weekday_columns == 6)|(test_weekday_columns == 7)]

smape_results = pd.DataFrame({i:np.ones(20) for i in range(6)})

for work in range(smape_results.shape[0]):
    for end in range(smape_results.shape[1]):

        test_pred_workday = pd.DataFrame({i:np.median(train_workday.iloc[:,1:].iloc[:,(-1)*work:], axis = 1) for i in test_workday.columns})
        test_pred_weekend = pd.DataFrame({i:np.median(train_weekend.iloc[:,1:].iloc[:,(-1)*end:], axis = 1) for i in test_weekend.columns})

        test_pred = pd.concat([test_pred_workday, test_pred_weekend], axis=1)


        test_predm = pd.melt(test_pred, value_vars=test_pred.columns[:-1], id_vars='Page')
        test_predm.sort_values(by = ['variable'], inplace = True)
        testm = pd.melt(test, value_vars=test.columns[1:], id_vars='Page')
        testm['pred_Visits'] = test_predm.value
        testm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

        smape_results.iloc[work, end] = pandas_smape(testm)
smape_results

,0,1,2,3,4,5
0,54.002589,56.502986,55.764152,55.386954,55.392198,55.422883
1,55.775309,58.275705,57.536871,57.159673,57.164917,57.195603
2,55.077931,57.578328,56.839494,56.462296,56.467540,56.498226
3,53.971921,56.472318,55.733484,55.356286,55.361530,55.392216
4,53.699531,56.199927,55.461093,55.083895,55.089139,55.119825
5,53.055447,55.555844,54.817009,54.439812,54.445056,54.475741
6,53.081970,55.582366,54.843532,54.466334,54.471578,54.502264
7,52.786949,55.287345,54.548511,54.171313,54.176557,54.207243
8,52.713621,55.214018,54.475184,54.097986,54.103230,54.133915
9,52.717657,55.218053,54.479219,54.102021,54.107265,54.137951
